In [2]:
import itertools
import pandas as pd
import numpy as np
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error

# Load your data
data = pd.read_csv("D:\\Stock_Anlysis\\btc_data.csv", parse_dates=['Date'])
data.set_index('Date', inplace=True)

data['value_diff'] = data['Close'].diff().dropna()

# Ensure the index is a DateTimeIndex and set frequency
data = data.asfreq('M')  # Assuming daily data, adjust as necessary

# Drop missing values
y = data['value_diff'].dropna()

# Define the p, d, q, P, D, Q, s parameters to take any value between 0 and 2
p = d = q = range(0, 3)
pdq = list(itertools.product(p, d, q))
seasonal_pdq = [(x[0], x[1], x[2], 12) for x in pdq]

# Grid search
best_aic = np.inf
best_params = None
for param in pdq:
    for param_seasonal in seasonal_pdq:
        try:
            mod = SARIMAX(y,
                          order=param,
                          seasonal_order=param_seasonal,
                          enforce_stationarity=False,
                          enforce_invertibility=False,
                          cov_type='approx')  # Change covariance type here
            results = mod.fit(disp=False)
            if results.aic < best_aic:
                best_aic = results.aic
                best_params = (param, param_seasonal)
        except Exception as e:
            print(f"Error with parameters {param} and {param_seasonal}: {e}")
            continue

print(f'Best SARIMA parameters: {best_params} with AIC: {best_aic}')

C:\Users\Home\AppData\Local\Temp\ipykernel_13972\582187897.py:14: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  data = data.asfreq('M')  # Assuming daily data, adjust as necessary
C:\Users\Home\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['cov_type']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
C:\Users\Home\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['cov_type']).Passing unknown keyword arguments will raise a TypeError beginning in version 0.15.
  warnings.warn(msg, FutureWarning)
C:\Users\Home\AppData\Roaming\Python\Python311\site-packages\statsmodels\tsa\statespace\representation.py:374: FutureWarning: Unknown keyword arguments: dict_keys(['cov_ty

Best SARIMA parameters: ((0, 2, 1), (0, 2, 2, 12)) with AIC: 8.0
